In [1]:
import cv2
import numpy as np
import glob
import math
import scipy
from scipy.spatial import distance
from scipy import signal
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics

In [2]:
#  !pip install unrar
#  !unrar x CASIA.rar

In [3]:
# def IrisLocalization(images):
#     # Convert all images to grayscale first
#     images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
#     # Create empty lists to store boundary and center
#     boundary = []
#     center = []
        
    
#     for img in images:
#         # Blurring the image
#         blurred = cv2.bilateralFilter(img,9,80,80)
#         img = blurred


#         # Estimating the center of the image
#         horizontal_mean = np.mean(img,axis = 0)
#         vertical_mean = np.mean(img,axis = 1)
#         # The indices with the min values
#         X_p = horizontal_mean.argmin()
#         Y_p = vertical_mean.argmin()

#         # Create a 120 * 120 region centered at the (X_p,Y_p)
#         circle_x = img[X_p-60:X_p+60]
#         circle_y = img[Y_p-60:Y_p+60]

#         # Update the center and estimate center of the pupil
#         horizontal_mean = np.mean(circle_y,axis = 0)
#         vertical_mean = np.mean(circle_x,axis = 0)

#         # Center of the 120*120 region
#         xp = horizontal_mean.argmin()
#         yp = vertical_mean.argmin()
#         # Define center of the pupil
#         pupil_center = (xp,yp)

#         copy_img = img.copy()
#         # Locate the pupil center and show it on the image with one point
#         cv2.circle(copy_img,(xp,yp),1,(255,0,0),2)

#         # Setting threshold
#         masked_img = cv2.inRange(img,0,70)
#         result = cv2.bitwise_and(img,masked_img)
#         # Apply Canny detector on masked image
#         img_edge = cv2.Canny(result, 100, 200)

#         # Apply Hough transform to the edged image
#         circle = cv2.HoughCircles(img_edge, cv2.HOUGH_GRADIENT, 10, 100)

#         min_dist = math.inf
#         for i in range(len(circle[0])):

#             c=(circle[0][i][0],circle[0][i][1])
#             dist = distance.euclidean(pupil_center, c)
#             if dist < min_dist:
#                 min_dist = dist
#                 k = circle[0][i]

#         img_orig = img.copy()
#         # Draw the inner boundary
#         cv2.circle(img_orig, (int(k[0]), int(k[1])), int(k[2]), (255, 0, 0), 2)

#         pupil = k
#         radius_pupil = int(k[2])

#         # Draw the outer boundary, with the inner boundary adding about 55-60 depending on different people  
#         cv2.circle(img_orig, (int(k[0]), int(k[1])), radius_pupil+55, (255, 0, 0), 2)

#         # plt.imshow(img_orig,cmap='gray')
#         boundary.append(img_orig)
#         center.append([int(k[0]),int(k[1]),int(k[2])])
#     return boundary,center

In [4]:
# def normalization(data, target_array):
#     img, centers = data[0], data[1]
#     # print(len(img), len(img[0]))
#     # for img in target:
#     #load pupil centers and radius of inner circles
#     x_pupil = centers[0]
#     y_pupil = centers[1]
#     radius_pupil = int(centers[2])
#     radius_iris = 55 # width of space between inner and outer boundary

#     def map_element(idx):
#       X, Y = idx[0], idx[1]
#       # print(idx)
#       # print(X, Y)

#       theta = 2 * np.pi * X/N
#       # adjacent side for pupil and iris 
#       adj_p = np.cos(theta) * radius_pupil
#       adj_i = np.cos(theta) * radius_iris

#       # opoosite side for pupil and iris
#       oppo_p = np.sin(theta) * radius_pupil
#       oppo_i = np.sin(theta) * radius_iris

#       # points on pupil circle: x_p(theta) and y_p(theta) from equation
#       x_p = int(np.round(x_pupil + adj_p))
#       y_p = int(np.round(y_pupil + oppo_p))

#       # points on iris circile: x_i(theta) and y_i(theta) from equation 
#       x_i = int(np.round(x_pupil + adj_i))
#       y_i = int(np.round(y_pupil + oppo_i))

#       # x & y (new projection) from equation
#       x = int(x_p + (x_i - x_p) * (Y/M))
#       y = int(y_p + (y_i - y_p) * (Y/M))

#       # x & y should not exceed image boundary i.e(320,280)
#       if y >= 320:
#           y = 319
#       if x >= 280:
#           x = 279
#       return img[x, y]


#     # print(rows.shape)
#     myfunc_vec = np.vectorize(map_element)
#     normalized = myfunc_vec(target_array)
#     return normalized
#     #return normalized

In [5]:
# def normalization(data):
#     img, centers = data[0], data[1]
#     # print(len(img), len(img[0]))
#     # for img in target:
#     #load pupil centers and radius of inner circles
#     x_pupil = centers[0]
#     y_pupil = centers[1]
#     radius_pupil = int(centers[2])
#     normalized = []
#     radius_iris = 55 # width of space between inner and outer boundary

#     M = 64
#     N = 512

#     for Y in range(M):
#         row_pix = []
        
#         for X in range(N):
#             theta = 2 * np.pi * X/N
#             # adjacent side for pupil and iris 
#             adj_p = np.cos(theta) * radius_pupil
#             adj_i = np.cos(theta) * radius_iris

#             # opoosite side for pupil and iris
#             oppo_p = np.sin(theta) * radius_pupil
#             oppo_i = np.sin(theta) * radius_iris

#             # points on pupil circle: x_p(theta) and y_p(theta) from equation
#             x_p = int(np.round(x_pupil + adj_p))
#             y_p = int(np.round(y_pupil + oppo_p))

#             # points on iris circile: x_i(theta) and y_i(theta) from equation 
#             x_i = int(np.round(x_pupil + adj_i))
#             y_i = int(np.round(y_pupil + oppo_i))

#             # x & y (new projection) from equation
#             x = int(x_p + (x_i - x_p) * (Y/M))
#             y = int(y_p + (y_i - y_p) * (Y/M))

#             # x & y should not exceed image boundary i.e(320,280)
#             if y >= 320:
#                 y = 319
#             if x >= 280:
#                 x = 279
    
#             row_pix.append(img[x, y])
#         normalized.append(np.array(row_pix))

#     return ImageEnhancement(np.array(normalized))
#     #return normalized

In [6]:
# def ImageEnhancement(normalized):
 
#     enhanced_normal = normalized.copy()

#     # range for row and column
#     nrow = int(normalized.shape[0]/32) # 2
#     ncol = int(normalized.shape[1]/32) # 16

#     # The following loop will first define each 32*32 region in normalized image
#     # and then perform histogram equalization in each divided region
    
#     for row in range(nrow):
#         for col in range(ncol):

#             # Devide normalized image into 32*32 regions
#             region = enhanced_normal[row*32 : (row+1)*32, col*32 : (col+1)*32]

#             # Perform histogram equalization for each 32*32 region
#             enhanced_normal[row*32 : (row+1)*32, col*32 : (col+1)*32] = cv2.equalizeHist(region)

#     return np.array(enhanced_normal)

In [7]:
# import cv2
# import numpy as np
# import glob
# import math
# import scipy
# from scipy.spatial import distance
# from scipy import signal
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# import matplotlib.pyplot as plt
# import pandas as pd
# from sklearn import metrics

# # Define the modualting function as mentioned in Ma's paper
# def M(x ,y, f):
#     res = np.cos(2 * np.pi * f * math.sqrt(x ** 2 + y ** 2))
#     return res

# # Define the Spatial filter
# def Gabor(x, y, dx, dy, f):
#     # print(x, y, dx, dy, f)
#     gabor = (1/(2 * np.pi * dx * dy)) * np.exp(-1/2 * (x ** 2/(dx ** 2) + y ** 2/(dy ** 2))) * M(x,y,f)
#     return gabor


# # Apply the defined filter to an 8 by 8 block 
# def block(dx, dy, f):
#     feature = np.zeros((8,8))
#     for i in range(8):
#         for j in range(8):
#             feature[i,j] = Gabor((-4+j),(-4+i),dx,dy,f)
#     return feature


# # The function inputs the two filtered image, calculates mean and std for each block of each channel, 
# # and returns the desired vector V with all means and stds appended to it.
# def get_vector(vector1,vector2):
#     feature_vec = []
#     # ranges are determined by 48/8 = 6 and 512/8 = 64
#     for i in range(6):
#         for j in range(64):
#             # Loop over each 8 by 8 block to get the feature
#             x1 = 8 * i
#             x2 = x1 + 8
#             y1 = 8 * j
#             y2 = y1 + 8
            
#             # Filtered image block
#             c1 = vector1[x1:x2,y1:y2]
#             c2 = vector2[x1:x2,y1:y2]
            
#             c1 = np.abs(c1)
#             c2 = np.abs(c2)
            
#             # Follow the calculation steps in Ma's paper
#             # Channel 1 mean and standard deviation
#             m1 = np.mean(c1)
#             sigma1 = np.mean(np.abs(c1-m1))
#             feature_vec.append(m1)
#             feature_vec.append(sigma1)
            
#             # Channel 2 mean and standard deviation
#             m2 = np.mean(c2)
#             sigma2 = np.mean(np.abs(c2-m2))
#             feature_vec.append(m2)
#             feature_vec.append(sigma2)
            
#     return feature_vec

# # Inputs a single normalized image
# def FeatureExtraction(enhanced_normal):
#     f = 2/3
#     # Get two channels using the parameters defined in paper
#     channel1 = block(3, 1.5, f)
#     channel2 = block(4, 1.5, f)
    
#     feature_vec = []
    

#     # enhanced_normal has length 64 and enhance_normal has length 512
#     # Define a 48 by 512 region as ROI
#     ROI = enhanced_normal[:48,:]

#     filtered1 = scipy.signal.convolve2d(ROI,channel1,mode='same')
#     filtered2 = scipy.signal.convolve2d(ROI,channel2,mode='same')
    
#     vector = get_vector(filtered1,filtered2)
#     feature_vec.append(vector)
#     # len(feature_vec) == 1536
#     return np.array(feature_vec).flatten()

In [8]:
# # def slice_per(source, step):
# #     return [np.array(source[i::step]).reshape(-1,1536) for i in range(step)]

# def LDA(df, target_dim):
#   # data = np.array(data)
#   mu = np.mean(df['feature'].values,axis=0).reshape(1536,1) # Mean vector mu --> Since the data has been standardized, the data means are zero 
#   mu_k = []

#   for i,idx in enumerate(np.unique(df['idx'])):
#       mu_k.append(np.mean(df[df['idx']==idx]['feature'],axis=0))
#   mu_k = np.array(mu_k).T

#   data_SW = []
#   Nc = []
#   for i,idx in enumerate(np.unique(df['idx'])):
#       a = np.array(np.array(df[df['idx']==idx]['feature'].values.tolist()).reshape(-1, 1536)-mu_k[:,i].reshape(1,1536))
#       data_SW.append(np.dot(a.T,a))
#       Nc.append(np.sum(df['idx']==idx))
#   SW = np.sum(data_SW,axis=0)

#   SB = np.dot(Nc*np.array(mu_k-mu),np.array(mu_k-mu).T)
#   eigval, eigvec = np.linalg.eig(np.dot(np.linalg.inv(SW),SB))

#   eigen_pairs = [[np.abs(eigval[i]),eigvec[:,i]] for i in range(len(eigval))]
#   eigen_pairs = sorted(eigen_pairs,key=lambda k: k[0],reverse=True)
#   w = np.hstack([eigen_pairs[i][1][:,np.newaxis].real for i in range(target_dim)]) # Select two largest

#   # 6. Transform the data with Y=X*w
#   # Y = data.dot(w)

#   return w



In [9]:
# images_train = [cv2.imread(file) for file in sorted(glob.glob('CASIA Iris Image Database (version 1.0)/*/1/*.bmp'))]

# boundary,center = IrisLocalization(images_train)

# bc_list = list(zip(boundary, center))

# # M = 64
# # N = 512
# # rows = np.empty((N, M), dtype=object)
# # for Y in range(M):
# #     for X in range(N):
# #         rows[X][Y] = (X, Y)



In [10]:
# normal_out = list(map(lambda x: normalization(x), bc_list))


In [11]:
# res=list(map(lambda p: getRotation(p, [-9,-6,-3,0,3,6,9]), np.array(normal_out.copy())))
# sliced_res = list(chunks(res,3))

# list_of_dataframes = []
# for i, eyes in enumerate(sliced_res):
#   # i from 0 to 108
#   for j, eye in enumerate(eyes):
#     # j from 0 to 2
#     degree = [-9,-6,-3,0,3,6,9]
#     img_idx = [j]*7
#     idx = [i]*7
#     df_tmp = pd.DataFrame({'idx':idx, 'img_idx':img_idx, 'degree':degree, 'data':eye})
#     list_of_dataframes.append(df_tmp)
# df = pd.concat(list_of_dataframes)


In [12]:
# df['feature']=df['data'].apply(lambda x: FeatureExtraction(x))

In [13]:
# w = LDA(df, 40)

In [14]:
# images_test = [cv2.imread(file) for file in sorted(glob.glob('CASIA Iris Image Database (version 1.0)/*/2/*.bmp'))]

# boundary_test,center_test = IrisLocalization(images_train)

# bc_list_test = list(zip(boundary_test, center_test))
# normal_out_test = list(map(lambda x: normalization(x), bc_list_test))

In [15]:
# sliced_res_test = list(chunks(normal_out_test, 4))

# list_of_dataframes_test = []
# for i, eyes in enumerate(sliced_res_test):
# # i from 0 to 107
#     for j, eye in enumerate(eyes):
#         # j from 0 to 3
#         img_idx = [j]
#         idx = [i]
#         df_tmp = pd.DataFrame({'idx':idx, 'img_idx':img_idx, 'data':[eye]})
#         list_of_dataframes_test.append(df_tmp)
      
# df_test = pd.concat(list_of_dataframes_test)
# df_test['feature']=df_test['data'].apply(lambda x: FeatureExtraction(x))


In [16]:
# clf = LinearDiscriminantAnalysis(n_components=40)
# clf.fit(list(df['feature'].values), list(df['idx'].values))
# df['new_feature']=df['feature'].apply(lambda x: clf.transform(x.reshape(1, -1)))
# df_test['new_feature']=df_test['feature'].apply(lambda x: clf.transform(x.reshape(1, -1)))

In [17]:
# df_test.head()

In [18]:
# import pickle

# with open('df_train.pkl', 'wb') as f:
#   pickle.dump(df, f)

# with open('df_test.pkl', 'wb') as f:
#   pickle.dump(df_test, f)

In [19]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [20]:
def compute_dis(test_feature, target_feature, norm):
  if norm == 1:
    return np.sum(np.abs(test_feature-target_feature))
  elif norm == 2:
    return np.sum(np.square(test_feature-target_feature))
  else:
    return np.multiply(test_feature.T,target_feature)/(np.linalg.norm(test_feature)*np.linalg.norm(target_feature))

In [21]:
def get_class(df_train, test_feature, label, norm):
  train_templates = df_train[label].values
  # print(len(train_templates))
  dis = list(map(lambda x: compute_dis(test_feature, x, norm),train_templates))
  min_idx = np.argmin(dis)
  # print(dis)
  # print(min_idx)
  return list(df_train['idx'].values)[min_idx]

In [22]:
from IrisLocalization import IrisLocalization
from IrisNormalization import IrisNormalization, getRotation
from ImageEnhancement import ImageEnhancement
from FeatureExtraction import FeatureExtraction
# from IrisMatching import IrisMatching
# from PerformanceEvaluation import PerformanceEvaluation

In [ ]:
images_train = [cv2.imread(file) for file in sorted(glob.glob('CASIA Iris Image Database (version 1.0)/*/1/*.bmp'))]
#running Localization, Normalization,Enhancement and Feature Extraction on all the training images
boundary_train,centers_train=IrisLocalization(images_train)
normalized_train=IrisNormalization(boundary_train,centers_train)
rotate_train = sum(list(map(lambda p: getRotation(p, [-9,-6,-3,0,3,6,9]), normalized_train.copy())), [])

enhanced_train=ImageEnhancement(rotate_train)
feature_vector_train=FeatureExtraction(enhanced_train)
print("Training data processed.")


'''TESTING'''

#reading the testing images from the CASIA dataset
images_test = [cv2.imread(file) for file in sorted(glob.glob('CASIA Iris Image Database (version 1.0)/*/2/*.bmp'))]

#running Localization, Normalization,Enhancement and Feature Extraction on all the testing images
boundary_test,centers_test=IrisLocalization(images_test)
normalized_test=IrisNormalization(boundary_test,centers_test)
enhanced_test=ImageEnhancement(normalized_test)
feature_vector_test=FeatureExtraction(enhanced_test)
print("Testing data processed.")

/Users/fred/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/fred/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [ ]:
sliced_res_train = list(chunks(feature_vector_train,21))

list_of_dataframes_train = []
for i, eyes in enumerate(sliced_res_train):
# i from 0 to 107
    sliced_eyes = list(chunks(eyes,7))
    for j, eye in enumerate(sliced_eyes):
        # j from 0 to 3
        degree = [-9,-6,-3,0,3,6,9]
        img_idx = [j]*7
        idx = [i]*7
        df_tmp = pd.DataFrame({'idx':idx, 'img_idx':img_idx, 'degree':degree, 'feature': eye })
        list_of_dataframes_train.append(df_tmp)
df_train = pd.concat(list_of_dataframes_train)

In [ ]:
df_train.head(30)

In [ ]:
sliced_res_test = list(chunks(feature_vector_test,4))

list_of_dataframes_test = []
for i, eyes in enumerate(sliced_res_test):
# i from 0 to 107
    for j, eye in enumerate(eyes):
        # j from 0 to 3
        img_idx = [j]
        idx = [i]
        df_tmp = pd.DataFrame({'idx':idx, 'img_idx':img_idx, 'feature':[np.array(eye)]})
        list_of_dataframes_test.append(df_tmp)
      
df_test = pd.concat(list_of_dataframes_test)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
clf = LDA(n_components=100)
clf.fit(list(df_train['feature'].values), list(df_train['idx'].values))
df_train['reduced_feature']=df_train['feature'].apply(lambda x: clf.transform(np.array(x).reshape(1, -1)))
df_test['reduced_feature']=df_test['feature'].apply(lambda x: clf.transform(np.array(x).reshape(1, -1)))

In [ ]:
df_test['class']=df_test['reduced_feature'].apply(lambda x: get_class(df_train.copy(), x, 'reduced_feature', 2))

In [ ]:
df_test

In [ ]:
df_test['correct'] = df_test.apply(lambda x: int(x['idx']==x['class']), axis=1)

In [ ]:
sum(df_test['correct'])